# Más temas de SQL


1. Consultas: GROUP BY, WHERE, ORDER BY ...

In [ ]:
import pandas as pd
import sqlite3 as sql 
from pym import tools as t

# 1. DISTINCT: se le aplica a una columna
# a. Valores unicos de esa columna
# b. Eliminar duplicados

# Selecciona TODAS las columnas de la tabla ventas
query = """
SELECT 
    --- Esto es un comentario en SQL
    * --- TODAS LAS COLUMNAS
FROM 
    ventas
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-02,Pc,EQV-772,34,883.16,30027.44,Zócalo
1,2025-01-02,Pc,VLQ-162,6,156.74,940.44,Coapa
2,2025-01-02,Plumones,BRF-523,19,8244.50,156645.50,Narvarte
3,2025-01-02,Cartulinas,LKI-773,41,738.70,30286.70,Coapa
4,2025-01-02,Uniformes,MHV-384,10,157.42,1574.20,Cuemanco
...,...,...,...,...,...,...,...
32995,2025-01-31,Plumones,JHZ-959,41,739.98,30339.18,Polanco
32996,2025-01-31,Cajas,JVZ-929,2,1167.60,2335.20,Coapa
32997,2025-01-31,Cajas,JWW-188,39,2872.38,112022.82,CU
32998,2025-01-31,Monitores,HDZ-745,2,5862.35,11724.70,Santa Fé


In [14]:
# Order by solito, el ordenamiento es de manera ascendente
query = """
SELECT 
    DISTINCT Fecha
FROM 
    ventas
--- Sentencia de ordenamiento
ORDER BY
    Fecha DESC
--- limit n, te muestra las primeras n filas
LIMIT 5
"""
t.consultar_bbdd(query)

,Fecha
0,2025-01-31
1,2025-01-30
2,2025-01-29
3,2025-01-28
4,2025-01-27


In [23]:
# Las 5 primeras ventas del 6 de enero en Polanco
# WHERE ---> filtros

# Seleccioname todas las columnas de la tabla ventas
# donde la sucursal sea Polando Y el dia sea el 6 de enero
#                               &
#                              and
# o ---> | ----> or
query = """
SELECT 
    Producto, 
    Cantidad,
    Total AS Total_Ventas
FROM 
    ventas
WHERE
    Sucursal = 'Polanco'
AND 
    Fecha = '2025-01-06'
LIMIT 5
"""
t.consultar_bbdd(query)

,Producto,Cantidad,Total_Ventas
0,Laptops,44,170588.44
1,Papel,22,36862.76
2,Cartulinas,38,32517.36
3,Pinceles,12,17186.88
4,Mouse,6,32351.40


In [ ]:
('Laptop', 44, 170000), ('Papel', 22, 3666)

Jerarquía de instrucciones en SQL
1. SELECT
2. FROM
3. WHERE
4. ORDER BY
5. LIMIT 5

Funciones de agregación:
1. SUM
2. MIN/MAX
3. AVG
4. COUNT

In [26]:
# CAST() ---> te convierte una columna al tipo de dato que le indiques
# CAST(NOM_COL AS TIPO_DATO_NUEVO)
query = """
SELECT 
    CAST(SUM(Total) AS INTEGER) AS Total_Ventas_Enero_Global
FROM 
    ventas
"""
t.consultar_bbdd(query)

,Total_Ventas_Enero_Global
0,2089119154


In [ ]:
2,089,119,154

1. Definir una función en Python
2. Subir a SQL

In [ ]:
def separador_miles(numero):
    """
    Da formato a un número insertando separadores de miles.

    Parámetros:
    ----------
    numero : int o float
        El número al que se le desea aplicar el formato con separadores de miles.

    Retorna:
    -------
    str
        El número formateado como cadena con comas separando los miles.
    """
    return "{:,}".format(numero)

sep_miles = lambda numero: "{:,}".format(numero)

print(separador_miles(1000))
print(sep_miles(1000))

1,000
1,000


In [ ]:
# Conectarse a la base de datos
con = sql.connect("ventas.db")
cursor = con.cursor()

# a. nombre de la funcion en SQL
# b. cuantos parametros tiene tu funcion
# c. pones la funcion de Python
con.create_function("separador_miles", 1, separador_miles)

query = """
SELECT 
    separador_miles(
        CAST(
            SUM(Total) 
        AS 
            INTEGER
        )
    ) AS Total_Ventas_Enero_Global
FROM 
    ventas
"""

# fetchone() ---> nos muestra solo un valor
consulta_01 = cursor.execute(query).fetchone()

# Guardamos cambios
con.commit()
# Cerramos la conexion
con.close()

consulta_01

('2,089,119,154',)

No puedes usar funciones creadas por ti mism@ con pandas, entonces debemos ver una alternativa

In [40]:


# Conectarse a la base de datos
con = sql.connect("ventas.db")
cursor = con.cursor()

# a. nombre de la funcion en SQL
# b. cuantos parametros tiene tu funcion
# c. pones la funcion de Python
con.create_function("separador_miles", 1, separador_miles)

query = """
SELECT 
    Producto, 
    Cantidad,
    Total AS Total_Ventas
FROM 
    ventas
WHERE
    Sucursal = 'Polanco'
AND 
    Fecha = '2025-01-06'
"""

# fetchone() ---> nos muestra solo un valor
# fetchall() ---> todas las filas disponibles
# fetchmany(n) ---> traeme los primeros n registros
consulta_02 = cursor.execute(query).fetchmany(5)

# Guardamos cambios
con.commit()
# Cerramos la conexion
con.close()

consulta_02

[('Laptops', 44, 170588.44),
 ('Papel', 22, 36862.76),
 ('Cartulinas', 38, 32517.36),
 ('Pinceles', 12, 17186.88),
 ('Mouse', 6, 32351.4)]

# Resumen de códigos

1. ``WHERE``: Realizar filtros. Va combinada usualmente con operadores lógicos (AND, OR).
2. ``ORDER BY``: Realizar el ordenamiento de la tabla con base en una columna. Por defecto se ordena de manera ascendente. ``DESC``, ordenas de manera descendente.
3. ``DISTINCT``: elimina duplicados, te arroja los valores únicos de una columna.
4. ``LIMIT N``: te muestra los primeros N registros
5. ``OFFSET``: salto de página

* ``LIMIT-OFFSET``: paginación

In [50]:
query = """
SELECT 
    *
FROM 
    ventas
LIMIT 15
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-02,Pc,EQV-772,34,883.16,30027.44,Zócalo
1,2025-01-02,Pc,VLQ-162,6,156.74,940.44,Coapa
2,2025-01-02,Plumones,BRF-523,19,8244.50,156645.50,Narvarte
3,2025-01-02,Cartulinas,LKI-773,41,738.70,30286.70,Coapa
4,2025-01-02,Uniformes,MHV-384,10,157.42,1574.20,Cuemanco
5,2025-01-02,Cajas,FYE-592,8,6179.40,49435.20,Centro
6,2025-01-02,Pinturas,MUG-932,18,900.28,16205.04,Centro
7,2025-01-02,Pc,ZHX-798,37,34.06,1260.22,Milpa Alta
8,2025-01-02,Baterías,MNO-935,39,542.51,21157.89,Polanco
9,2025-01-02,Pc,BLW-128,47,3238.40,152204.80,Milpa Alta


In [49]:
query = """
SELECT 
    *
FROM 
    ventas
LIMIT 5
OFFSET 10
"""
t.consultar_bbdd(query)

,Fecha,Producto,Clave,Cantidad,Precio,Total,Sucursal
0,2025-01-02,Cajas,JPA-156,16,1062.55,17000.80,Milpa Alta
1,2025-01-02,Libretas,UYB-527,18,53.97,971.46,Polanco
2,2025-01-02,Bolsas,QGN-994,20,3426.98,68539.60,Zócalo
3,2025-01-02,Mouse,GGG-691,30,7552.39,226571.70,Cuemanco
4,2025-01-02,Pegamento,XVN-745,24,1032.45,24778.80,CU
